# Docker Container Orchestration

On sleeping in demo code:

Putting a 0.1 second sleep in demo code closesly simulates an actual request. While a program is blocked on a network I/O call, it basically _sleeps_. 

* Performance testing code
    * `ab` Apache Bench
    * `httpping` Ping, using HTTP
        * In the real world, you'd use something like Data Dog or New Relic
    * `vmstat`
    * `curl` !
    
* Scaling w/ Docker-Compose
    * Doesn't support _load balancing_
    * If your program doesn't need to manage routing connections, that's OK
        * Example: Worker processes that consume/prouce from the same I/O
    * Use a load balancer container
        * jpetazzo/hamba `
    * Us a script to parse docker-compose.yml and make changes
    
* Sysadmin'ing Docker containers
    * Use a linked container. This avoids customizing the app container, so you can reuse it across environments and tool changes
    * Logging
        * Run a container with a volume; `-v /logs`. 
        * `sh -c "grep WARN /logs/*.log"` <--- The `sh -c` prevents shell expansion of the wildcard.
        * Log shipper: 
            * `tail -f` on logs
            * Forwards them to..._wherever_
        * Ship to the same location/file, and handle logging outside of the app (12-Factor app much?)
        * Logging to Syslog
            * Write everything to stdout
            * Use `--log-driver syslog` Docker option
            * Breaks `docker logs`
        * Logging to JSON files
            * Will eventually use up all space
            * Docker plans on writing a better solution at some point (handle log rotation themselves)
        * Docker Logstash Driver (One day!)

* On Docker security
    * Upgrading the Docker daemon
        * Steps
            1. Kill all containers
            1. Stop daemon
            1. Upgrade Docker
            1. Start Docker
            1. Start all containers
        * Basically like upgrading your kernel
        * Will get better
            1. Decouple containers from Docker
                * What??? 
            1. CRIU (Checkpoint-and-Restore-In-Userland)
                1. Freeze to disk container
                    1. Maybe ship them to a different unit
                1. ...upgrade Docker daemon...
                1. Resume containers.
    * Upgrading container images
        * In order of importance:
            1. Upgrade base images first
            1. Then downloaded packages
            1. Re-pull base image
                * Makes sure we upgrade any second- or third-order packages related to the OS upgrade
            1. Rebuild
            1. Restart container
            
* On network analysis
    * Usually you'd just use `ngrep`, `tcpdump`, `wireshark`, etc.
    * But you can _share_ a network namespace
        * `... --net container:myredis ...` 
        * `ss`
            * Looking at sockets on the shared network
        * `ngrep`
            * `ngrep -tpd eth0 -Wbyline wallet tcp and host 54.69.170.174`
            * Watch `eth0`, scanning `tcp` packets for `wallet`, but only from/to host `54.69.170.174`
            
            

## Static & Dynamic Scheduling

* On static
    * Let's say small is less than 100 VMs
    * You probably don't need dynamic scheduling! 
* Dynamic
    * _Requires_ high availability considerations, key-value stores (and by extension, service discovery)

* Mesos
    * > "Meh" for exclusively Docker systems
    * Might be better if you have a mix of VMs and Docker containers

* Kubernetes
    * Greenfield; doesn't need to support legacy systems
    * Replication controller manages resource usage and HA
    * Pods => Groups of containers sharing network/storage
    * Tags over tree (Flat over nested)
        * Can add/remove tags from Pods to activate or deactivate them while live.
    * Cons
        * Complex service discovery and network
        * GCE-based
        * Differs from Docker in API, requiring learning
        * Very-focused on the Ops experience, too young for the Developer experience.
        
* Swarm
    * Shares Docker API, 100%
    * Masks Docker cluster behind appearance of single host
    * Good for Jenkins slaves, Grid computing
    * _Not_ good (yet) for Container links, Compose

* PaaS
    * Easier on Devs (Simpler interface
    * Harder on Ops (Now you have to manage a PaaS!) 
    

## Security

* Two major security concenrs
    1. Keeping Alice and Jane's containers from breaking into each other while running on the same host
        * Mostly a PaaS concern
    1. Make sure our `docker run <name>` actually runs `<name>`
        * Make sure no one is injecting a bad image
        * Not a huge deal! Breaking into a worker that lives alongside a web server in a container is really no different from a VM break-in